In [1]:
import pandas as pd
import asyncio
from typing import List, Dict, Any, Optional, Tuple
from sqlalchemy.ext.asyncio import AsyncSession
from pathlib import Path
from datetime import datetime
import os
import sys

current_file_path = Path(os.getcwd()) # Jupyter Notebook에서는 os.getcwd()가 현재 노트북 파일이 위치한 디렉토리를 반환합니다.
project_root = current_file_path.parent.parent # app/services/ -> app -> project_root
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from app.core.logger import get_logger
from app.utils.excel_utils import read_excel_file
from app.core.constants.customs import CustomsTypeConfig as Config
from app.repositories.customs_repository import ExportImportStatByCountryRepository

logger = get_logger()

In [2]:
regex = Config.MAJOR_CATEGORY_REGEX
regex

'^(1|2)\\.\\s*'

In [3]:
async def _export_preprocess_data(
    df: pd.DataFrame
)-> pd.DataFrame:
    
    major_regex = Config.MAJOR_CATEGORY_REGEX
    sub_regex = Config.SUB_CATEGORY_REGEX
    
    # 1. 성질명 컬럼에서 major/sub 정규표현식에 해당하는 행만 남김
    is_major = df[Config.EXCEL_CATEGORY].astype(str).str.match(major_regex)
    is_sub = df[Config.EXCEL_CATEGORY].astype(str).str.match(sub_regex)
    df = df[is_major | is_sub].copy()

    # 2. "카. 기 타"를 "카. 경공업품(기타)"로 변경, "바. 기 타"를 "바. 중화학 공업품(기타)"로 변경
    def replace_k_other(category: str) -> str:
        if isinstance(category, str) and category.strip().startswith("카. 기 타"):
            return "카. 경공업품(기타)"
        elif isinstance(category, str) and category.strip().startswith("바. 기 타"):
            return "바. 중화학 공업품(기타)"
        
        return category

    df[Config.EXCEL_CATEGORY] = df[Config.EXCEL_CATEGORY].apply(replace_k_other)

    # 3. major/sub prefix(예: "1. ", "가. ") 제거
    def remove_prefix(category: str) -> str:
        if not isinstance(category, str):
            return category
        # major prefix 제거
        
        category = pd.Series(category).str.replace(major_regex, "", regex=True).iloc[0]
        # sub prefix 제거
        category = pd.Series(category).str.replace(sub_regex, "", regex=True).iloc[0]
        return category.strip()

    df[Config.EXCEL_CATEGORY] = df[Config.EXCEL_CATEGORY].apply(remove_prefix)

    return df

In [4]:
async def _transform_country_name(
        df: pd.DataFrame,
        repository: ExportImportStatByCountryRepository
)-> pd.DataFrame:
    try:
        # 관세청 국가명 -> ISO 코드 매핑
        country_names = await repository.get_country_name_mapping()
        # ISO 코드 -> 무보 국가명 매핑
        country_iso_names = await repository.get_country_iso_mapping()


        # 관세청 국가명 -> ISO 코드 변환 함수 정의
        
        def map_korean_country_to_iso(korean_country_name: str)-> str:
            # korean_country_name = korean_country_name.strip()
            return country_names.get(korean_country_name, None)
        
        # ISO 코드 -> 무보 국가명으로 변환
        def map_iso_to_mubo_country(iso_code: str)-> str:
            # iso_code = iso_code.strip()
            return country_iso_names.get(iso_code, None)
        
        df[Config.TEMP_ISO_CODE] = df[Config.EXCEL_COUNTRY].apply(map_korean_country_to_iso)
        df[Config.EXCEL_COUNTRY] = df[Config.TEMP_ISO_CODE].apply(map_iso_to_mubo_country)

        # 매핑되지 않은 국가(ISO 코드가 None이거나 국가명이 None인 경우) 제거
        df = df[df[Config.TEMP_ISO_CODE].notnull() & df[Config.EXCEL_COUNTRY].notnull()].reset_index(drop=True)
        
        logger.info(f"국가명 변환 완료: {len(df)}행")
        return df

    except Exception as e:
        logger.error(f"Error transforming country name: {e}")
        raise e

In [5]:
async def _create_final_output(
        df: pd.DataFrame
)-> pd.DataFrame:
    try:
        # 최종 형태로 데이터 변환
        final_df = df.rename(columns=Config.get_final_column_mapping())

        # 정렬 (예: 첫 번째 컬럼은 오름차순, 두 번째 컬럼은 내림차순)
        sort_columns = Config.get_sort_columns()
        ascending = [True, False]  # 필요에 따라 동적으로 지정
        final_df = final_df.sort_values(by=sort_columns, ascending=ascending)

        return final_df
    except Exception as e:
        logger.error(f"Error creating final output: {e}")
        raise e

In [6]:
async def process_data(
    file_path: str,
    file_name: str,
    dbprsr: AsyncSession = None,
    dbpdtm: AsyncSession = None,
    replace_all: bool = True,
)-> pd.DataFrame:
    
    file_path = Path(file_path,file_name)
    logger.info(f"관세청 수출/수입품 파일 처리 시작:{file_path}")
    
    raw_df = await read_excel_file(file_path, header_cols=Config.get_header_columns())

    preprocess_df = await _export_preprocess_data(raw_df)

    return preprocess_df

In [7]:
file_path = "/appdata/storage/research/original"
file_name = "6-2. 수출 실적(성질별+국가별)_20250711.xlsx"
file_fullpath = Path(file_path, file_name)

In [8]:
raw_df = await read_excel_file(file_fullpath, header_cols=Config.get_header_columns())
raw_df

/home/younpark/anaconda3/envs/py310/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/younpark/anaconda3/envs/py310/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
2025-07-18 10:36:39.513 | INFO     | app.utils.excel_utils:read_excel_file:66 - 엑셀 파일 읽기 완료: 21285행


,기간,수출입구분,국가,성질명,중량,금액
1,2023,수출,가나,1. 식료 및 직접소비재,"27,075.3","27,238"
2,2023,수출,가나,- 기타 육류 및 조제품,0.1,1
3,2023,수출,가나,- 어패류와 조제품,"26,753.5","25,312"
4,2023,수출,가나,(기타 어패류),"26,753.3","25,312"
5,2023,수출,가나,(생 선 묵),0.2,1
...,...,...,...,...,...,...
21281,2023,수출,홍콩,(콘테이너),540.2,478
21282,2023,수출,홍콩,(자전거),0.0,2
21283,2023,수출,홍콩,(기 타),39.2,"7,415"
21284,2023,수출,홍콩,바. 기 타,"18,441.5","586,563"


In [9]:
preprocess_df = await _export_preprocess_data(raw_df)
preprocess_df[preprocess_df[Config.EXCEL_COUNTRY] == '가나']

,기간,수출입구분,국가,성질명,중량,금액
1,2023,수출,가나,식료 및 직접소비재,"27,075.3","27,238"
14,2023,수출,가나,원료 및 연료,"3,186.7","2,429"
19,2023,수출,가나,섬유사,46.4,121
21,2023,수출,가나,직 물,168.9,192
24,2023,수출,가나,기타 섬유제품,"1,711.1","4,271"
29,2023,수출,가나,의 류,32.7,310
35,2023,수출,가나,목제품,"1,367.8",480
38,2023,수출,가나,"가죽, 고무 및 신발류",681.5,666
42,2023,수출,가나,귀금속 및 보석류,0.0,1
44,2023,수출,가나,기타 비금속 광물제품,0.0,0


In [10]:
from app.db.base import get_main_db

async def main():
    async for dbprsr in get_main_db():
        expimp_repository = ExportImportStatByCountryRepository(dbprsr)
        transformed_df = await _transform_country_name(preprocess_df, expimp_repository)
        break
        
    return transformed_df
    
transformed_df = await main()
transformed_df

2025-07-18 10:36:41.141 | INFO     | __main__:_transform_country_name:29 - 국가명 변환 완료: 3351행


,기간,수출입구분,국가,성질명,중량,금액,ISO코드
0,2023,수출,가나,식료 및 직접소비재,"27,075.3","27,238",GH
1,2023,수출,가나,원료 및 연료,"3,186.7","2,429",GH
2,2023,수출,가나,섬유사,46.4,121,GH
3,2023,수출,가나,직 물,168.9,192,GH
4,2023,수출,가나,기타 섬유제품,"1,711.1","4,271",GH
...,...,...,...,...,...,...,...
3346,2023,수출,홍콩,철강제품,"39,979.2","198,836",HK
3347,2023,수출,홍콩,기계류와 정밀기기,"13,368.3","784,646",HK
3348,2023,수출,홍콩,"전기, 전자제품","19,255.2","19,606,345",HK
3349,2023,수출,홍콩,수송장비,"638,882.9","677,777",HK


In [11]:
final_df = await _create_final_output(transformed_df)
final_df

,impexp_year,impexp_flag,impexp_nation_nm,impexp_item_nm,impexp_item_weight,impexp_item_money,impexp_nation_code
12,2023,수출,가나,화공품,"61,461.4","76,233",GH
16,2023,수출,가나,수송장비,"27,036.3","33,063",GH
0,2023,수출,가나,식료 및 직접소비재,"27,075.3","27,238",GH
14,2023,수출,가나,기계류와 정밀기기,"4,184.8","14,913",GH
15,2023,수출,가나,"전기, 전자제품","1,624.9","9,636",GH
...,...,...,...,...,...,...,...
3337,2023,수출,홍콩,기타 섬유제품,"2,205.6","31,836",HK
3343,2023,수출,홍콩,"완구, 운동용구 및 악기",518.6,"25,352",HK
3339,2023,수출,홍콩,목제품,509.9,"5,337",HK
3335,2023,수출,홍콩,섬유사,303.0,"4,891",HK


In [12]:
# df = await process_data(
#     file_path = file_path,
#     file_name = file_name,
# )

# df[df[Config.EXCEL_COUNTRY] == '가나']